<a href="https://colab.research.google.com/github/Flamanjaune/ppi-time-series-analysis/blob/main/ppi_time_series_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Article

## Packages

In [ ]:
!pip install -q altair==5.2.0 \
                 matplotlib==3.8.2 \
                 numpy>=1.26.4 \
                 pandas==2.2.2 \
                 pmdarima>=2.0.4 \
                 Rbeast==0.1.23 \
                 scipy>=1.14.0 \
                 seaborn==0.13.2 \
                 statsmodels==0.14.1

In [ ]:
# Calculs scientifiques
import numpy as np
import pandas as pd
from scipy import stats

# Visualisation
import altair as alt
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator, DateFormatter
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Modèles statistiques et séries temporelles
import pmdarima as pm
import Rbeast as rb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

## Data

### Upload File

In [ ]:
from google.colab import files
uploaded = files.upload()

### Load Dataframe

In [ ]:
df = pd.read_csv("PPI_data_final.csv")
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='M')
df

### Plotting

In [ ]:
# PPI pills sales
plt.style.use('default')
fig, ax = plt.subplots(figsize=(12,7))

plt.plot(df['Date'].values, df['Y_PANTOPRAZOLE'].values, label='Pantoprazole')
plt.plot(df['Date'].values, df['Y_ESOMEPRAZOLE'].values, label='Esomeprazole')
plt.plot(df['Date'].values, df['Y_OMEPRAZOLE'].values, label='Omeprazole')
plt.plot(df['Date'].values, df['Y_LANSOPRAZOLE'].values, label='Lansoprazole')
plt.plot(df['Date'].values, df['Y_RABEPRAZOLE'].values, label='Rabeprazole')
plt.plot(df['Date'].values, df['Y_DEXLANSOPRAZOLE'].values, label='Dexlansoprazole')

plt.title('Comparison of PPI pills sales rates by molecules')
plt.ylabel('PPI pills sales rates per 100.000 habitants')

plt.legend()

plt.show()

In [ ]:
# Intravenous PPI sales
plt.style.use('default')
fig, ax = plt.subplots(figsize=(12,7))

# Plot bounce rate data
ax.scatter(df['Date'], df['Y_PPI_IV'], facecolors='k', edgecolors='k', label='IV PPI data', linewidths=1,s=10)

plt.title('Intravenous PPI sales rate in Switzerland over time')
plt.ylabel("Intravenous PPI sales rate per 100'000 habitants");

plt.legend()

plt.show()

## Data Analysis

In [ ]:
# convert the Purchase Date to datetime
df['Date'] = pd.to_datetime(df['Date'])
# add a column for Year
df['Year'] = df['Date'].dt.year
# print the dataframe
df

### Adfuller Test for Stationarity

This test is used to decide whether the data are stationary or not.
- If the data are stationary, we set d = 0.
- If the data are not stationary, we need to analyze the model in greater depth.

In [ ]:
def adf_test(dataset):
    dftest = adfuller(dataset, autolag = 'AIC')
    print("1. ADF : ",dftest[0])
    print("2. P-Value : ", round(dftest[1],6))
    print("3. Num Of Lags : ", dftest[2])
    print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
    print("5. Critical Values :")
    for key, val in dftest[4].items():
       print("\t",key, ": ", val)
    if dftest[0] < dftest[4]["5%"]:
        print ("Reject Ho - Time Series is Stationary")
    else:
        print ("Failed to Reject Ho - Time Series is Non-Stationary")

adf_test(df.Y)

### Seasonality

Looking at the ACF chart, we can see that the autocorrelation trend is decreasing, with peaks every 6 months.

We'll therefore choose 6 or 12 months as the seasonality.

In [ ]:
lag_acf = 50
lag_pacf = 50

f, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
plot_acf(df.Y,lags=lag_acf, ax=ax[0],alpha=.05);
plot_pacf(df.Y,lags=lag_pacf, ax=ax[1],alpha=.05);

We can confirme this hypothesis by de-trending the data

In [ ]:
df = pd.read_csv("PPI_data_final.csv")

df_trend = df - df.rolling(15).mean()

# Drop the NaN values
df_trend = df_trend.dropna()
df_trend[df_trend < 0] = 15000
# Create figure and subplots
lag_acf = 36
lag_pacf = 36

f, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
plot_acf(df_trend.Y,lags=lag_acf, ax=ax[0],alpha=.05);
plot_pacf(df_trend.Y,lags=lag_pacf, ax=ax[1],alpha=.05);

According to the de-trend acf and pacf, we set m=12 for the seasonality.

### Auto ARIMA

It is now possible to use the auto-arima function to define the missing terms (p,q,P,Q,D).

In [ ]:
df = pd.read_csv("PPI_data_final.csv")
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='M')
# fit stepwise auto-ARIMA
# aic
stepwise_fit_aic = pm.auto_arima(df.Y, start_p=1, start_q=1, max_p=5, max_q=5,d=0,
                             start_P=1, D=None, start_Q=1, max_P=5, max_D=1, max_Q=5, max_order=None, seasonal=True,m=12,
                             maxiter=20, trace=True,alpha=0.05, information_criterion = "aic",
                             return_valid_fits = True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True);  # set to stepwise
# bic
stepwise_fit_bic = pm.auto_arima(df.Y, start_p=1, start_q=1, max_p=5, max_q=5,d=0,
                             start_P=1, D=None, start_Q=1, max_P=5, max_D=1, max_Q=5, max_order=None, seasonal=True,m=12,
                             maxiter=20, trace=True,alpha=0.05, information_criterion = "bic",
                             return_valid_fits = True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True);  # set to stepwise
# hqic
stepwise_fit_hqic = pm.auto_arima(df.Y, start_p=1, start_q=1, max_p=5, max_q=5,d=0,
                             start_P=1, D=None, start_Q=1, max_P=5, max_D=1, max_Q=5, max_order=None, seasonal=True,m=12,
                             maxiter=20, trace=True,alpha=0.05, information_criterion = "hqic",
                             return_valid_fits = True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True);  # set to stepwise
# oob
stepwise_fit_oob = pm.auto_arima(df.Y, start_p=1, start_q=1, max_p=5, max_q=5,d=0,
                             start_P=1, D=None, start_Q=1, max_P=5, max_D=1, max_Q=5, max_order=None, seasonal=True,m=12,
                             maxiter=20, trace=True,alpha=0.05, information_criterion = "oob", out_of_sample_size=12,  # Specify non-zero holdout
                             return_valid_fits = True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True);  # set to stepwise

Create a table with the best parameters

In [ ]:
d = {'aic': stepwise_fit_aic[0:5], 'bic': stepwise_fit_bic[0:5], 'hqic': stepwise_fit_hqic[0:5], 'oob': stepwise_fit_oob[0:5] }
df = pd.DataFrame(data=d)
df

Now we can perform ljungbox and normal test for each set of parameters.

In [ ]:
df = pd.read_csv("PPI_data_final.csv")
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='M')

aic_ljungbox = []
aic_normal = []
bic_ljungbox = []
bic_normal = []
hqic_ljungbox = []
hqic_normal = []
oob_ljungbox = []
oob_normal = []

#aic
for i in range(5):
  mod = stepwise_fit_aic[i]
  p,d,q = mod.get_params(deep=False)["order"]
  P,D,Q,s = mod.get_params(deep=False)['seasonal_order']
  arima_results = ARIMA(df["Y"], df[["T","D","P"]], order=(p,d,q),seasonal_order=(P,D,Q,s)).fit()
  ljungbox = sm.stats.acorr_ljungbox(arima_results.resid, lags=[24], return_df=True)
  normal = stats.normaltest(arima_results.resid)
  aic_ljungbox.append(ljungbox["lb_pvalue"][24])
  aic_normal.append(normal.pvalue)

#bic
for i in range(5):
  mod = stepwise_fit_bic[i]
  p,d,q = mod.get_params(deep=False)["order"]
  P,D,Q,s = mod.get_params(deep=False)['seasonal_order']
  arima_results = ARIMA(df["Y"], df[["T","D","P"]], order=(p,d,q),seasonal_order=(P,D,Q,s)).fit()
  ljungbox = sm.stats.acorr_ljungbox(arima_results.resid, lags=[24], return_df=True)
  normal = stats.normaltest(arima_results.resid)
  bic_ljungbox.append(ljungbox["lb_pvalue"][24])
  bic_normal.append(normal.pvalue)

#hqic
for i in range(5):
  mod = stepwise_fit_hqic[i]
  p,d,q = mod.get_params(deep=False)["order"]
  P,D,Q,s = mod.get_params(deep=False)['seasonal_order']
  arima_results = ARIMA(df["Y"], df[["T","D","P"]], order=(p,d,q),seasonal_order=(P,D,Q,s)).fit()
  ljungbox = sm.stats.acorr_ljungbox(arima_results.resid, lags=[24], return_df=True)
  normal = stats.normaltest(arima_results.resid)
  hqic_ljungbox.append(ljungbox["lb_pvalue"][24])
  hqic_normal.append(normal.pvalue)

#oob
for i in range(5):
  mod = stepwise_fit_oob[i]
  p,d,q = mod.get_params(deep=False)["order"]
  P,D,Q,s = mod.get_params(deep=False)['seasonal_order']
  arima_results = ARIMA(df["Y"], df[["T","D","P"]], order=(p,d,q),seasonal_order=(P,D,Q,s)).fit()
  ljungbox = sm.stats.acorr_ljungbox(arima_results.resid, lags=[24], return_df=True)
  normal = stats.normaltest(arima_results.resid)
  oob_ljungbox.append(ljungbox["lb_pvalue"][24])
  oob_normal.append(normal.pvalue)

In [ ]:
d = {'aic_ljungbox': aic_ljungbox, 'aic_normal': aic_normal, 'bic_ljungbox': bic_ljungbox, 'bic_normal': bic_normal,
     'hqic_ljungbox': hqic_ljungbox, 'hqic_normal': hqic_normal, 'oob_ljungbox': oob_ljungbox, 'oob_normal': oob_normal}
df_test = pd.DataFrame(data=d)
df_test

From the table, we can deduce that the best parameters are the 1st and the 2nd from the bic measure (the ones on the list to accept $H_0$ for both the residuals and the normality). We choose the 2nd one, as it visually better fits our data)

In [ ]:
stepwise_fit_bic[1]

### Test

In [ ]:
df = pd.read_csv("PPI_data_final.csv")
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='ME')
p = 3
d = 0
q = 0
P = 2
D = 0
Q = 0
s = 12
arima_results = ARIMA(df["Y"], df[["T","D","P"]], order=(p,d,q),seasonal_order=(P,D,Q,s)).fit()
print(arima_results.summary())

### Plot

In [ ]:
start = 66
end = 132
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='M')
intervention_date = df['Date'][start-1]

predictions = arima_results.get_prediction(0,end-1)
summary = predictions.summary_frame(alpha=0.05)

arima_cf = ARIMA(df["Y"][0:start], df["T"][0:start], order=(p,d,q),seasonal_order=(P,D,Q,s)).fit()

# Model predictions means
y_pred = predictions.predicted_mean

# Counterfactual mean and 95% confidence interval
y_cf = arima_cf.get_forecast(end-start, exog=df["T"][start:]).summary_frame(alpha=0.05)

# Plot section
plt.style.use('default')
fig, ax = plt.subplots(figsize=(12,7))
ax.set_ylim([150000, 480000])

# Plot bounce rate data
ax.scatter(df["Date"], df["Y"], facecolors='k', edgecolors='k', label="PPI data", linewidths=1,s=10)

# Plot model mean bounce prediction
ax.plot(df["Date"][:start], y_pred[0:start], 'C0-', label="Model prediction")
ax.plot(df["Date"][start-1:], y_pred[start-1:], 'C0-')

# Plot counterfactual mean bounce rate with 95% confidence interval
ax.plot(df["Date"][start:], y_cf["mean"], 'C1-', label="Counterfactual")
ax.fill_between(df["Date"][start:], y_cf['mean_ci_lower'], y_cf['mean_ci_upper'], color='C1', alpha=0.2, label="Counterfactual 95% CI");


# Plot line marking intervention moment
ax.axvline(x = intervention_date, color = 'C3', linestyle='--', label = 'Intervention (May 2014)')
ax.legend(loc='upper left')
plt.title('PPI pills sales rate in Switzerland over time')
#plt.xlabel("Date")
plt.ylabel("PPI pills sales rate per 100'000 habitants");

It is possible to check the quality of the parameters set for the model.

Ljung-Box test to check if autocorrelation exists in a time series:
- H0: The residuals are independently distributed.

- H1: The residuals are not independently distributed; they exhibit serial correlation.

In [ ]:
sm.stats.acorr_ljungbox(arima_results.resid, lags=[24], return_df=True)

Here lb_value > 0.05 so we can conclude that the residuals are independently distributed.

And we can check if the residuals are normally distributed.

In [ ]:
from scipy import stats
stats.normaltest(arima_results.resid)

In [ ]:
arima_results.plot_diagnostics(figsize=(14,10));

Here's what we can deduce:
- No correlation in residuals (top-left plot)
- The residuals follows a normal distribution (top-right and bottom-left plot)
- Low correlations in residuals (bottom-right plot)

## Rbeast

### BMA Analysis for PPI pills


In [ ]:
df = pd.read_csv("PPI_data_final.csv")
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='ME')
df

In [ ]:
data, date = df.Y, df.Date
o = rb.beast(data, start=2009, deltat=1/12, period = 1.0)

In [ ]:
# Plot section
rb.plot(o);

In [ ]:
rb.print(o)

In [ ]:
print(dir(o.trend.cpOccPr))

In [ ]:
# Assuming o.time is the time array that aligns with cpOccPr
cp_time = o.time
cp_occ_pr = o.trend.cpOccPr
df = pd.DataFrame({
    'cp_time': cp_time,
    'cp_occ_pr': cp_occ_pr,
    'Date': pd.date_range(start='2009-01-01', periods=len(cp_time), freq='ME')
})
print(df.head(24))

# Define the interval start and end
interval_start = 2014
interval_end = 2016
intervention_date = df['Date'][65]

# Filter the DataFrame for the desired interval
filtered_df = df[(df['cp_time'] >= interval_start) & (df['cp_time'] < interval_end)]
filtered_df1 = df[(df['cp_time'] >= 2013.85) & (df['cp_time'] < 2016)]

# Sum the probabilities for the filtered interval
sum_probs = filtered_df['cp_occ_pr'].sum()
print("Sum of changepoint probabilities over the interval:", sum_probs)
print("We have a", sum_probs * 100, "% chance that the change occurred between Jan-2014 and Jan-2016")

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['cp_occ_pr'], label='Changepoint Occurrence Probability')
plt.fill_between(filtered_df1['Date'], filtered_df1['cp_occ_pr'], color='skyblue', alpha=0.4, label=f'AUC from 01/2014 to 01/2016: {sum_probs:.2f}')
plt.ylabel('Probability')
plt.title('Changepoint Occurrence Probability For PPI Pills Sales Rate Over Time')
plt.axvline(x=intervention_date, color='C3', linestyle='--', label='Intervention (May 2014)')
plt.legend()
plt.ylim(-0.005, 0.107)
plt.show()


In [ ]:
# Define the interval start and end
interval_start = 2014
interval_end = 2016


# o.time is a NumPy array with time points
# o.trend.cpOccPr is a NumPy array with corresponding probabilities

# Find indices where time falls within the interval
interval_indices = np.where((o.time >= interval_start) & (o.time < interval_end))

# Extract the probabilities for these indices
interval_probs = o.trend.cpOccPr[interval_indices]

# Sum the probabilities
sum_probs = np.sum(interval_probs)

print("Sum of changepoint probabilities over the interval:", sum_probs)
print("We have a", sum_probs*100, "% chance that the change occured between Jan-2014 and Dec-2016")

### BMA Analysis for PPI Infusion bottles

In [ ]:
df = pd.read_csv("PPI_data_final.csv")
df['Date'] = pd.date_range(start='2009-01-01', periods=len(df), freq='ME')

In [ ]:
data, date = df.Y_PPI_IV, df.Date
o = rb.beast(data, start=2009, deltat=1/12, period = 1.0)

In [ ]:
# Plot section
rb.plot(o);

In [ ]:
rb.print(o)

In [ ]:
print(dir(o.trend.cpOccPr))

In [ ]:
# Assuming o.time is the time array that aligns with cpOccPr
cp_time = o.time
cp_occ_pr = o.trend.cpOccPr
df = pd.DataFrame({
    'cp_time': cp_time,
    'cp_occ_pr': cp_occ_pr,
    'Date': pd.date_range(start='2009-01-01', periods=len(cp_time), freq='ME')
})
print(df.head(24))

# Define the interval start and end
interval_start = 2014
interval_end = 2016
intervention_date = df['Date'][65]

# Filter the DataFrame for the desired interval
filtered_df = df[(df['cp_time'] >= interval_start) & (df['cp_time'] < interval_end)]
filtered_df1 = df[(df['cp_time'] >= 2013.85) & (df['cp_time'] < 2016)]

# Sum the probabilities for the filtered interval
sum_probs = filtered_df['cp_occ_pr'].sum()
print("Sum of changepoint probabilities over the interval:", sum_probs)
print("We have a", sum_probs * 100, "% chance that the change occurred between Jan-2014 and Jan-2016")

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['cp_occ_pr'], label='Changepoint Occurrence Probability')
plt.fill_between(filtered_df1['Date'], filtered_df1['cp_occ_pr'], color='skyblue', alpha=0.4, label=f'AUC from 01/2014 to 01/2016: {sum_probs:.2f}')
plt.ylabel('Probability')
plt.title('Changepoint Occurrence Probability For PPI Infusion Bottles Over Time')
plt.axvline(x=intervention_date, color='C3', linestyle='--', label='Intervention (May 2014)')
plt.legend()
plt.ylim(-0.005, 0.107)
plt.show()

In [ ]:
# Define the interval start and end
interval_start = 2014
interval_end = 2016


# o.time is a NumPy array with time points
# o.trend.cpOccPr is a NumPy array with corresponding probabilities

# Find indices where time falls within the interval
interval_indices = np.where((o.time >= interval_start) & (o.time < interval_end))

# Extract the probabilities for these indices
interval_probs = o.trend.cpOccPr[interval_indices]

# Sum the probabilities
sum_probs = np.sum(interval_probs)

print("Sum of changepoint probabilities over the interval:", sum_probs)
print("We have a", sum_probs*100, "% chance that the change occured between Jan-2014 and Dec-2016")